In [ ]:
import sys
from pathlib import Path

from aiida import load_profile
from ase.io import read
from tb_hamiltonian.workflows import compute_bands

In [ ]:
sys.tracebacklimit = None

In [ ]:
_ = load_profile()

# Define parameters


In [ ]:
# Control parameters
debug = False

# Initial structure
structure_label = "BLG"
input_path = Path("examples/BLG/AB/rectangular")
initial_structure = read(input_path / "POSCAR", format="vasp")

# Distances
distances = [0.0, 1.425, 2.468, 2.850]

# Supercell parameters
nx = 1  # number of repetitions in x direction
ny = 1  # number of repetitions in y direction
nz = 1  # number of repetitions in z direction

# Hamiltonian parameters
hopping_parameters = [0.0, -2.7, 0.0, -0.27]
interlayer_coupling = 0.33
nearest_neighbor = 1  # degree of nearest neighbors | don't use 0!

# Potential parameters
alpha = [1.0, 0.5]
onsite_term = 0.0
potential_type = "null"
potential_params = {
    "amplitude": 1.0,
    # "width": 0.5,
    # "height": 0.5,
}

# Band structure parameters
band_params = {
    "high_sym_points": {
        "Γ": (0.00000, 0.00000, 0.00000),
        "P": (0.00000, 0.33333, 0.00000),
        "X": (0.00000, 0.50000, 0.00000),
        "W": (0.50000, 0.50000, 0.00000),
        "Y": (0.50000, 0.00000, 0.00000),
    },
    "path": "Γ P X W Y Γ W",
    "points_per_segment": 100,
    "use_sparse_solver": False,
    "sparse_solver_params": {"k": 3, "sigma": 1e-8},
    "save_data": True,
}

# Metadata

For more scheduler options, see

https://aiida.readthedocs.io/projects/aiida-core/en/latest/topics/calculations/usage.html#options


In [ ]:
h, m, s = 1, 0, 0

NODES = 1
TASKS_PER_NODE = 1
CPUS_PER_TASK = 1

CONDA_ENV = ""
assert CONDA_ENV, "Please set the CONDA_ENV variable"


local_metadata = {
    "options": {
        "prepend_text": f"""
source $(conda info --base)/etc/profile.d/conda.sh && conda activate {CONDA_ENV}
        """,
        "withmpi": False,
        "resources": {
            "num_machines": NODES,
            "num_mpiprocs_per_machine": TASKS_PER_NODE,
            "num_cores_per_mpiproc": CPUS_PER_TASK,
        },
        "max_wallclock_seconds": h * 3600 + m * 60 + s,
    }
}

REMOTE_ENV_PATH = ""
assert REMOTE_ENV_PATH, "Please set the REMOTE_ENV_PATH variable"

remote_metadata = {
    "options": {
        "prepend_text": f"""
module load cray-python
source {REMOTE_ENV_PATH}/bin/activate
        """,
        "withmpi": False,
        "resources": {
            "num_machines": NODES,
            "num_mpiprocs_per_machine": TASKS_PER_NODE,
            "num_cores_per_mpiproc": CPUS_PER_TASK,
        },
        "max_wallclock_seconds": h * 3600 + m * 60 + s,
    }
}

input_metadata = {
    "build_hamiltonian": {
        "computer": "localhost",
        "metadata": {
            "options": {
                **local_metadata["options"],
                "additional_retrieve_list": ["POSCAR"],
            }
        },
    },
    "apply_onsite_term": {
        "computer": "localhost",
        "metadata": local_metadata,
    },
    "get_band_structure": {
        "computer": "localhost",
        "metadata": local_metadata,
    },
}

# Build and submit work graph


In [ ]:
wg = compute_bands(
    structure_label=structure_label,
    initial_structure=initial_structure,
    repetitions=[nx, ny, nz],
    distances=distances,
    nearest_neighbor=nearest_neighbor,
    hopping_parameters=hopping_parameters,
    interlayer_coupling=interlayer_coupling,
    potential_type=potential_type,
    potential_params=potential_params,
    onsite_term=onsite_term,
    alpha=alpha,
    band_params=band_params,
    metadata=input_metadata,
)

In [ ]:
wg.submit()

# Monitoring


In [ ]:
! verdi process status $wg.pk

# Analysis

In [ ]:
# from aiida.orm import load_node
# from tb_hamiltonian.utils import BandStructure

# wg_node = load_node(wg.pk)
# band_calculation = wg_node.called[-1]
# band_structure: BandStructure = band_calculation.outputs.result.value
# band_structure.plot()

# Write structure POSCAR locally

In [ ]:
# from aiida.orm import FolderData

# local_filename = "POSCAR"
# retrieved: FolderData = wg_node.called[0].outputs.retrieved
# with retrieved.open("POSCAR", mode="r") as f, open(local_filename, "w") as g:
#     g.write(f.read())